3d CNN Classification of 3d MNIST Dataset taken from Kaggle https://www.kaggle.com/datasets/daavoo/3d-mnist


In [1]:
import numpy as np 
import h5py
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#loading dataset

with h5py.File('full_dataset_vectors.h5', 'r') as dataset:
  xtrain, xtest = dataset["X_train"][:], dataset["X_test"][:]
  ytrain, ytest = dataset["y_train"][:], dataset["y_test"][:]
xtrain = np.array(xtrain)
xtest = np.array(xtest)
print('train shape:', xtrain.shape)
print('test shape:', xtest.shape)
xtrain = xtrain.reshape(xtrain.shape[0], 16, 16, 16, 1)
xtest = xtest.reshape(xtest.shape[0], 16, 16, 16, 1)
ytrain, ytest = to_categorical(ytrain, 10), to_categorical(ytest, 10)

train shape: (10000, 4096)
test shape: (2000, 4096)


In [3]:
model = Sequential()
model.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(16,16,16,1),bias_initializer=Constant(0.01)))
model.add(layers.Conv3D(32,(3,3,3),activation='relu',bias_initializer=Constant(0.01)))
model.add(layers.MaxPooling3D((2,2,2)))
model.add(layers.Conv3D(64,(3,3,3),activation='relu'))
model.add(layers.Conv3D(64,(2,2,2),activation='relu'))
model.add(layers.MaxPooling3D((2,2,2)))
model.add(layers.Dropout(0.6))
model.add(layers.Flatten())
model.add(layers.Dense(256,'relu'))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(128,'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 14, 14, 14, 32)    896       
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 12, 12, 32)    27680     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 6, 6, 6, 32)      0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 4, 4, 4, 64)       55360     
                                                                 
 conv3d_3 (Conv3D)           (None, 3, 3, 3, 64)       32832     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 1, 1, 64)      0         
 3D)                                                    

In [4]:
model.compile(Adam(0.001),'categorical_crossentropy',['accuracy'])
model.fit(xtrain,ytrain,epochs=200,batch_size=32,verbose=1,validation_data=(xtest,ytest),callbacks=[EarlyStopping(patience=15)])

Epoch 1/200
313/313 [==============================] - 16s 11ms/step - loss: 2.3015 - accuracy: 0.1066 - val_loss: 2.2846 - val_accuracy: 0.1320
Epoch 2/200
313/313 [==============================] - 3s 10ms/step - loss: 2.1864 - accuracy: 0.1698 - val_loss: 2.0488 - val_accuracy: 0.2320
Epoch 3/200
313/313 [==============================] - 3s 10ms/step - loss: 2.0129 - accuracy: 0.2441 - val_loss: 1.7852 - val_accuracy: 0.3090
Epoch 4/200
313/313 [==============================] - 3s 10ms/step - loss: 1.8506 - accuracy: 0.2973 - val_loss: 1.6165 - val_accuracy: 0.3810
Epoch 5/200
313/313 [==============================] - 3s 10ms/step - loss: 1.7411 - accuracy: 0.3401 - val_loss: 1.5286 - val_accuracy: 0.4310
Epoch 6/200
313/313 [==============================] - 3s 10ms/step - loss: 1.6815 - accuracy: 0.3675 - val_loss: 1.5080 - val_accuracy: 0.4320
Epoch 7/200
313/313 [==============================] - 3s 10ms/step - loss: 1.6222 - accuracy: 0.3784 - val_loss: 1.4384 - val_accuracy

In [5]:
_, acc = model.evaluate(xtrain, ytrain)
print('training accuracy:', str(round(acc*100, 2))+'%')
_, acc = model.evaluate(xtest, ytest)
print('testing accuracy:', str(round(acc*100, 2))+'%')

313/313 [==============================] - 1s 5ms/step - loss: 0.5477 - accuracy: 0.8190
training accuracy: 81.9%
63/63 [==============================] - 0s 5ms/step - loss: 1.1304 - accuracy: 0.6515
testing accuracy: 65.15%
